
## <center> DBSCAN Clustering Algorithm </center>

<div style="text-align: justify">Density-based spatial clustering of applications with noise (DBSCAN) is a data clustering algorithm proposed by Martin Ester, Hans-Peter Kriegel, Jörg Sander and Xiaowei Xu in 1996. It is a density-based clustering non-parametric algorithm: given a set of points in some space, it groups together points that are closely packed together (points with many nearby neighbors), marking as outliers points that lie alone in low-density regions (whose nearest neighbors are too far away). DBSCAN is one of the most common clustering algorithms and also most cited in scientific literature.</div> 

<img src="https://upload.wikimedia.org/wikipedia/commons/a/af/DBSCAN-Illustration.svg" width=300>

<div style="text-align: justify">In this diagram, $minPts = 4$. Point A and the other red points are core points, because the area surrounding these points in an ε radius contain at least 4 points (including the point itself). Because they are all reachable from one another, they form a single cluster. Points B and C are not core points, but are reachable from A (via other core points) and thus belong to the cluster as well. Point N is a noise point that is neither a core point nor directly-reachable.</div>

<p/>

<div style="text-align: justify">The DBSCAN algorithm can be abstracted into the following steps:</div> 


1. Find the points in the ε (eps) neighborhood of every point, and identify the core points with more than minPts neighbors.
2. Find the connected components of core points on the neighbor graph, ignoring all non-core points.
3. Assign each non-core point to a nearby cluster if the cluster is an ε (eps) neighbor, otherwise assign it to noise.

<div style="text-align: justify"> A naive implementation of this requires storing the neighborhoods in step 1, thus requiring substantial memory. The original DBSCAN algorithm does not require this by performing these steps for one point at a time.</div>  


<img src="https://scikit-learn.org/stable/_images/sphx_glr_plot_cluster_comparison_001.png">

### 1. Importing Libs

In [4]:
# Matrix and quaternions manipulations
import numpy as np
from pyquaternion import Quaternion

In [5]:
# Data clustering
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler

In [6]:
# Visualization
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.tools import make_subplots
init_notebook_mode(connected=True)

### 2. Functions

In [7]:
def trace3D(PointCloud, color, size, name):
    trace = go.Scatter3d(
        x=PointCloud[:,0],
        y=PointCloud[:,1],
        z=PointCloud[:,2],
        name=name,
        mode='markers',
        marker=dict(
            size=size,
            line=dict(
                color=color,
                width=0.5
            ),
            opacity=0.8
        )
    )
    return trace

In [8]:
def rotateQuaternion(x,y,z,q):
    xyz = np.transpose(np.vstack([x,y,z]))
    listxyz = []
    for i in range(xyz.shape[0]):
        v = xyz[i,:]
        v_prime = q.rotate(v)
        listxyz.append(v_prime)
    xyz_prime = np.vstack(listxyz)
    return xyz_prime[:,0], xyz_prime[:,1], xyz_prime[:,2]

### 3. Generate Data

In [9]:
# Generate XY Grid
initX = 0.0
totalX = 20
endX = 10

initY = 0.0
totalY = 10
endY = 10

listX = []
for i in range(totalY):
    listX = listX + np.linspace(initX, endX, totalX).tolist()
x = np.array(listX)

listY = []
listValY = np.linspace(initY, endY, totalY)
for val in listValY:
    listY = listY + [val for i in range(totalX)]
y = np.array(listY)

print(x.shape)
print(y.shape)

(200,)
(200,)


In [10]:
# Getting Z from plane Equation
# Plane ax + by + cz = d ==> z = (1/c)(d - ax -by)
a = 0.0
b = 1.0
c = 1.0
d = 1.0

z = (1.0/c)*(d - a*x - b*y)
z_noise = z + np.random.rand(len(z))

In [32]:
numperpi = 3.141592654
np.set_printoptions(suppress=True) # Suppress insignificant values for clarity
q = Quaternion(axis=[1, 0, 0], angle=numperpi/4.0) # Rotate 0 about x=y=z
x_prime, y_prime, z_prime = rotateQuaternion(x,y,z_noise,q)
z_prime = z_prime + np.array([5]*len(z_prime))

In [33]:
X1 = np.transpose(np.vstack([x, y, z]))                   
X2 = np.transpose(np.vstack([x_prime, y_prime, z_prime]))
labels_true = np.array(([0]*X1.shape[0]) + ([1]*X2.shape[0]))
X = np.concatenate([X1,X2], axis=0)
print(X.shape)
print(labels_true.shape)

(400, 3)
(400,)


In [34]:
fig = make_subplots(rows=1, cols=1, specs=[[{'is_3d': True}]])

trace = trace3D(X, color='red', size=6, name='Source')

fig.append_trace(trace, 1, 1)

iplot(fig, filename='simple-3d-scatter')

This is the format of your plot grid:
[ (1,1) scene1 ]



### 4. Clustering: DBSCAN

In [38]:
X = StandardScaler().fit_transform(X)
# #############################################################################
# Compute DBSCAN
db = DBSCAN(eps=0.5, min_samples=10).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_
print(labels)

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1 -1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1

In [39]:
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(labels_true, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(labels_true, labels))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

Estimated number of clusters: 2
Estimated number of noise points: 3
Homogeneity: 1.000
Completeness: 0.947
V-measure: 0.973
Adjusted Rand Index: 0.985
Adjusted Mutual Information: 0.947
Silhouette Coefficient: 0.274


### 5. Visualizing clusters: cores, rechables and outliers

In [43]:
# Plot result
fig = make_subplots(rows=1, cols=1, specs=[[{'is_3d': True}]])
unique_labels = set(labels)
for k in unique_labels:
    if k == -1:
        # Black used for noise.
        color = 'black'
        name1 = 'Outlier'
        name2 = 'Outlier'
    else:
        color = 'red'
        name1 = 'Core'
        name2 = 'Rechable'

    class_member_mask = (labels == k)

    xy_core = X[class_member_mask & core_samples_mask]
    #ax.scatter(xy[:, 0], xy[:, 1], xy[:, 2], 'bo')

    xy_proximity = X[class_member_mask & ~core_samples_mask]
    #ax.scatter(xy[:, 0], xy[:, 1], xy[:, 2], 'bo')
    
    trace_core = trace3D(xy_core, color=color, size=6, name=name1)
    trace_proximity = trace3D(xy_proximity, color=color, size=3, name=name2)
    
    fig.append_trace(trace_core, 1, 1)
    fig.append_trace(trace_proximity, 1, 1)

iplot(fig, filename='simple-3d-scatter')

This is the format of your plot grid:
[ (1,1) scene1 ]

